In [3]:
import spacy
import pandas as pd
nlp= spacy.load('en_core_web_sm')


In [5]:
data_true= pd.read_csv('resources/True.csv')
data_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
data_true= data_true.drop(columns=['title', 'date', 'subject'])

In [22]:
data_true['label']= 'true'
data_true.head(5)

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,true
1,WASHINGTON (Reuters) - Transgender people will...,true
2,WASHINGTON (Reuters) - The special counsel inv...,true
3,WASHINGTON (Reuters) - Trump campaign adviser ...,true
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,true


In [6]:
data_false= pd.read_csv('resources/Fake.csv')
data_false.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [13]:
data_false= data_false.drop(columns=['date', 'title', 'subject'])

In [21]:
data_false['label']= 'false'
data_false.head(5)

,text,label
0,Donald Trump just couldn t wish all Americans ...,false
1,House Intelligence Committee Chairman Devin Nu...,false
2,"On Friday, it was revealed that former Milwauk...",false
3,"On Christmas day, Donald Trump announced that ...",false
4,Pope Francis used his annual Christmas Day mes...,false


In [24]:
df= pd.concat([data_false, data_true], axis= 0)
df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,false
1,House Intelligence Committee Chairman Devin Nu...,false
2,"On Friday, it was revealed that former Milwauk...",false
3,"On Christmas day, Donald Trump announced that ...",false
4,Pope Francis used his annual Christmas Day mes...,false


In [25]:
df.shape

(44898, 2)

In [26]:
df.label.value_counts()

false    23481
true     21417
Name: label, dtype: int64

In [27]:
# false = 0 true 1
df.label = df.label.apply(lambda x: 1 if x == 'true' else 0)

In [28]:
df.head(5)

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [29]:
df.label.value_counts()

0    23481
1    21417
Name: label, dtype: int64

In [48]:
def preprocess(text):
    doc= nlp(text)
    filter= []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            filter.append(token.lemma_)
    return ' '.join(filter)         

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

Without PreProcessing

In [33]:
x_train, x_test, y_train, y_test= train_test_split(df.text, df.label, test_size= 0.2, stratify= df.label, random_state= 2300)

In [38]:
#BOW MODEL (1 GRAM)
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [40]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96      4696
           1       0.95      0.95      0.95      4284

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980



In [43]:
#(2 GRAM) Model
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('nb', MultinomialNB())])

In [44]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4696
           1       0.97      0.97      0.97      4284

    accuracy                           0.97      8980
   macro avg       0.97      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980



In [45]:
#(3 GRAM) Model
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 3))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 3))),
                ('nb', MultinomialNB())])

In [46]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4696
           1       0.98      0.97      0.98      4284

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



With preprocessing

In [50]:
df.head(5)

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [51]:
df['text_new']= df.text.apply(preprocess)

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(df.text_new, df.label, test_size= 0.2, stratify= df.label)

In [ ]:
#BOW MODEL (1 GRAM)
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

In [ ]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
#(2 GRAM) Model
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

In [ ]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
#(3 GRAM) Model
from sklearn.naive_bayes import MultinomialNB

clf= Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 3))),
    ('nb', MultinomialNB())
])

clf.fit(x_train, y_train)

In [ ]:
y_pred= clf.predict(x_test)
print(classification_report(y_test, y_pred))